In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
PROJECT_ID=d_project_config['gcp-project-name']

In [ ]:
PROJECT_ID

In [ ]:
bq_client.list_datasets()

In [ ]:
datasets = list(bq_client.list_datasets())
table_refs = []



In [ ]:
datasets

In [ ]:

if datasets:
    print("Datasets in project {}:".format(PROJECT_ID))
    for dataset in datasets:
        # print("\t{}".format(dataset.dataset_id))
        tables = bq_client.list_tables(dataset)
        for table in tables:
            # print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))
            table_refs.append("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

else:
    print("{} project does not contain any datasets.".format(PROJECT_ID))

In [ ]:
table_refs

In [ ]:
# divg-team-v02-pr-5de84a



In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
table_refs_1=['divgpras-pr-579355.SHS.SMHM_Deact_Mar2023']

In [ ]:
prefix = 'divg-team-v02-pr-5de84a.adc_backup.'
for table_ref in table_refs:
    # table_name=prefix+table_ref.split(".")[-1]
    # bq_client.create_table(table_name)
    # config = bigquery.job.LoadJobConfig()
    # config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    Query=''' CREATE TABLE `divg-team-v02-pr-5de84a.adc_backup.{destination_table}`
    CLONE `divgpras-pr-579355.{source_dataset}.{soruce_table}`; '''.format(destination_table=table_ref.split(".")[-1] ,source_dataset=table_ref.split(".")[-2], soruce_table= table_ref.split(".")[-1])
    print(Query)
    bq_client.query(Query)


    # df_ucar=extract_bq_data(bq_client,sql=Query)
    # bq_table_instance = client.load_table_from_dataframe(df_ucar, table_ref, job_config=config)


In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''

CREATE TABLE
`divg-team-v02-pr-5de84a.adc_backup.ADC_ADT_mapping`
CLONE `divgpras-pr-579355.SHS.SMHM_Deact_Mar2023`;

'''

In [ ]:
DF=extract_bq_data(bq_client, sql=Query,)